In [16]:
import sys
from pathlib import Path

# Adiciona o diretório raiz do projeto ao path do Python
# Isso permite que o notebook encontre o módulo 'src'
project_root = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from datetime import timedelta, date
import asyncio
from tavily import AsyncTavilyClient
from dotenv import load_dotenv, find_dotenv
from src.config.settings import TAVILY_QUERIES, TAVILY_API_KEY


load_dotenv(find_dotenv())

def fetch_dates():
    today = date.today()
    previous_month = today - timedelta(days=90)

    return (str(today), str(previous_month))


# Verifica se a API key foi carregada
if not TAVILY_API_KEY:
    print("⚠️ TAVILY_API_KEY não encontrada. Verifique o arquivo .env")
else:
    tavily_client = AsyncTavilyClient(TAVILY_API_KEY)
    date_today, date_today_minus_90_days = fetch_dates()
    print(f"Hoje: {date_today}")
    print(f"Mês anterior: {date_today_minus_90_days}")
    print("✅ Tavily client inicializado com sucesso!")

Hoje: 2025-12-08
Mês anterior: 2025-09-09
✅ Tavily client inicializado com sucesso!


In [26]:
async def fetch_and_gather():
    queries = TAVILY_QUERIES[:3]

    try:
        responses = await asyncio.gather(
            *(
                tavily_client.search(
                    q, 
                    max_results=1, 
                    start_date=date_today_minus_90_days,
                    end_date=date_today,
                    country="brazil"
                ) for q in queries
            ), return_exceptions=True
        )

        for response in responses:
            if isinstance(response, Exception):
                print(f"Search query failed: {response}")  
            elif response["results"][0]["score"] < 0.75:
                print(f"tema pesquisado é irrelevante: {response["results"][0]["url"]}")
            else:
                print(response)                
    except Exception as e:
        print(f"Error during search queries: {e}")

In [27]:
await fetch_and_gather()

tema pesquisado é irrelevante: https://www.instagram.com/p/DOwKyHlj1DM/?hl=en
{'query': 'Dados recentes sobre a morosidade do judiciário brasileiro e seus impactos financeiros nas empresas', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.instagram.com/p/DPmJ_QYDmRk/', 'title': 'A Justiça brasileira é frequentemente criticada quanto ...', 'content': 'A Justiça brasileira é frequentemente criticada quanto à morosidade. Uma análise comparativa entre Brasil e Europa revela um cenário surpreendente: apesar', 'score': 0.9998902, 'raw_content': None}], 'response_time': 0.0, 'request_id': '8b3ed327-47b9-4d10-b51e-dcaa377829a6'}
tema pesquisado é irrelevante: https://fasadv.com.br/pt/bra/global-reach/brasil/expertise/trabalhista-e-previdenciario
